<a href="https://colab.research.google.com/github/blythecarr/NLP_ToxicCommentDetector/blob/main/STI202303367_WindyPangestuti_AI_NLP_ToxicCommentDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import library

In [1]:
import pandas as pd
import nltk
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel

2. Load dataset

In [2]:
df = pd.read_excel("/content/Toxic Comment Moderation Dataset.xlsx")

3. Ambil kolom penting dan rename

In [3]:
df = df[['comment_text', 'is_toxic']].rename(columns={'comment_text': 'text', 'is_toxic': 'label'})
df['label'] = df['label'].astype(int)

4. Split data

In [4]:
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

5. TF-IDF + PassiveAggressiveClassifier

In [5]:
tfidf = TfidfVectorizer(lowercase=True, stop_words='english', max_features=2000)
X_tr = tfidf.fit_transform(X_train)
X_va = tfidf.transform(X_val)

pac = PassiveAggressiveClassifier(max_iter=50, random_state=42)
pac.fit(X_tr, y_train)

y_pred = pac.predict(X_va)

print("🔎 Hasil Model TF-IDF + PAC")
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred, target_names=['Benign', 'Toxic']))

🔎 Hasil Model TF-IDF + PAC
Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

      Benign       0.94      0.96      0.95       138
       Toxic       0.91      0.89      0.90        72

    accuracy                           0.93       210
   macro avg       0.93      0.92      0.93       210
weighted avg       0.93      0.93      0.93       210



6. BERT EMBEDDING + PassiveAggressiveClassifier

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(texts):
    enc = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        outputs = bert(**enc)
    return outputs.pooler_output.numpy()

X_tr_bert = get_embeddings(X_train)
X_va_bert = get_embeddings(X_val)

pac_bert = PassiveAggressiveClassifier(max_iter=50, random_state=42)
pac_bert.fit(X_tr_bert, y_train)
y_pred_bert = pac_bert.predict(X_va_bert)

print("\n🔎 Hasil Model BERT + PAC")
print("Accuracy:", accuracy_score(y_val, y_pred_bert))
print(classification_report(y_val, y_pred_bert, target_names=['Benign', 'Toxic']))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🔎 Hasil Model BERT + PAC
Accuracy: 0.9380952380952381
              precision    recall  f1-score   support

      Benign       0.96      0.95      0.95       138
       Toxic       0.90      0.92      0.91        72

    accuracy                           0.94       210
   macro avg       0.93      0.93      0.93       210
weighted avg       0.94      0.94      0.94       210



7. Uji coba prediksi komentar baru

In [7]:
def predict_text(text, method='tfidf'):
    if method == 'tfidf':
        x = tfidf.transform([text])
        pred = pac.predict(x)[0]
    elif method == 'bert':
        emb = get_embeddings(pd.Series([text]))
        pred = pac_bert.predict(emb)[0]
    return "Toxic" if pred == 1 else "Benign"

print("\n🧪 Contoh Uji Komentar")
sample_texts = [
    "Stop being so stupid."
]

for text in sample_texts:
    print(f"\nKomentar: {text}")
    print("Prediksi TF-IDF + PAC  ➜", predict_text(text, method='tfidf'))
    print("Prediksi BERT + PAC    ➜", predict_text(text, method='bert'))


🧪 Contoh Uji Komentar

Komentar: Stop being so stupid.
Prediksi TF-IDF + PAC  ➜ Toxic
Prediksi BERT + PAC    ➜ Toxic
